In [1]:
import cv2
import numpy as np
import tensorflow as tf
import math
import time

In [2]:
model = tf.keras.models.load_model("CNNHANDWRITTEN.model" , compile = False)

W0113 03:38:44.833328 14284 deprecation.py:506] From C:\Users\Manav Khorasiya\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0113 03:38:44.837094 14284 deprecation.py:506] From C:\Users\Manav Khorasiya\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0113 03:38:44.854743 14284 deprecation.py:506] From C:\Users\Manav Khorasiya\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\resource_variable

In [3]:
def prepare(ima):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = ima*255
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE*3, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,3)

In [4]:
def alpha(prediction):
    if(prediction==0):
        prediction='A'
    elif(prediction==1):
        prediction='B'
    elif(prediction==2):
        prediction='C'
    elif(prediction==3):
        prediction='D'
    elif(prediction==4):
        prediction='G'
    elif(prediction==5):
        prediction='L'
    elif(prediction==6):
        prediction='M'
    elif(prediction==7):
        prediction='N'
    elif(prediction==8):
        prediction='O'
    elif(prediction==9):
        prediction='P'
    elif(prediction==10):
        prediction='R'
    elif(prediction==11):
        prediction='S'
    elif(prediction==12):
        prediction='T'
    elif(prediction==13):
        prediction='U'
    elif(prediction==14):
        prediction='V'
    elif(prediction==15):
        prediction='Z'
    else:
        prediction='nothing'
    return prediction

def callback(x):
    pass

In [5]:
cv2.namedWindow('image1')
cap=cv2.VideoCapture(0)
ilowH = 0
ihighH = 255
ilowS = 0
ihighS = 255
ilowV = 0
ihighV = 255
cv2.createTrackbar('lowH1', 'image1', ilowH, 255, callback)
cv2.createTrackbar('highH1', 'image1', ihighH, 255, callback)
cv2.createTrackbar('lowS1', 'image1', ilowS, 255, callback)
cv2.createTrackbar('highS1', 'image1', ihighS, 255, callback)
cv2.createTrackbar('lowV1', 'image1', ilowV, 255, callback)
cv2.createTrackbar('highV1', 'image1', ihighV, 255, callback)
while True:
    try:
        ret,frame=cap.read()
        ilowH1 = cv2.getTrackbarPos('lowH1', 'image1')
        ihighH1 = cv2.getTrackbarPos('highH1', 'image1')
        ilowS1 = cv2.getTrackbarPos('lowS1', 'image1')
        ihighS1 = cv2.getTrackbarPos('highS1', 'image1')
        ilowV1 = cv2.getTrackbarPos('lowV1', 'image1')
        ihighV1 = cv2.getTrackbarPos('highV1', 'image1')
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        low1=np.array([ilowH1,ilowS1,ilowV1])
        high1=np.array([ihighH1,ihighS1,ihighV1])
        img_mask1=cv2.inRange(hsv,low1,high1)
        output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
        new1 = cv2.cvtColor(output1, cv2.COLOR_BGR2GRAY)
        dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
        cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
        M=cv2.moments(dist)
        cX = (int(M["m10"] / M["m00"]))
        cY = (int(M["m01"] / M["m00"]))
        cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)
        cv2.imshow("image1",frame)
        cv2.imshow('dist',dist)
        cv2.imshow("image",output1)
    except ZeroDivisionError:
        pass
    if (cv2.waitKey(1) == 13):
                break
cv2.destroyAllWindows()
cap.release()
i=1000

In [8]:
cap=cv2.VideoCapture(0)
start=time.time()
black=np.zeros((output1.shape[0],output1.shape[1]))
j=1
while True:
    try:
        ret,frame=cap.read()
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        low1=np.array([ilowH1,ilowS1,ilowV1])
        high1=np.array([ihighH1,ihighS1,ihighV1])
        img_mask1=cv2.inRange(hsv,low1,high1)
        output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
        new1 = cv2.cvtColor(output1, cv2.COLOR_BGR2GRAY)
        dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
        cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
        M=cv2.moments(dist)
        cX = (int(M["m10"] / M["m00"]))
        cY = (int(M["m01"] / M["m00"]))
        cv2.circle(frame, (cX, cY), 5, (255, 252, 255), -1)
        cv2.circle(black, (cX, cY), 5, (255, 255, 255), -1)
#         if(j==1) :
#             cv2.putText(black,"start",(cX,cY),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),lineType=cv2.LINE_AA)
#         else :
        if(j!=1):
            cv2.line(black,(sX,sY),(cX,cY),(255,255,255),5)
        cv2.imshow("image1",frame)
        cv2.imshow("image",output1)
        cv2.imshow("path",black)
        j=j+1
        sX=cX
        sY=cY
        end=time.time()
        diff=end-start
        if (diff>7):
            black=cv2.flip(black,1)
            cv2.imwrite("Handwritten using motion tracking/"+str(i)+".jpg",black)
            img = cv2.imread("Handwritten using motion tracking/"+str(i)+".jpg")
            prediction = model.predict([prepare(new1)])
            prediction=np.argmax(prediction)
            x1=alpha(prediction)
            #x1=1
            cv2.putText(img,str(x1),(60,60),cv2.FONT_HERSHEY_SIMPLEX,3.0,(0,255,255),lineType=cv2.LINE_AA)
            cv2.imshow('new',img)
            black=np.zeros((output1.shape[0],output1.shape[1]))
            i=i+1
            start=time.time()
            print("hi")
    except ZeroDivisionError:
        pass
    if (cv2.waitKey(1) == 13):
                break
cv2.destroyAllWindows()
cap.release()

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
